<a href="https://colab.research.google.com/github/IsuruBoyagane15/vue4logs-baseline/blob/master/vue4logs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
project_directory = "Vue4Logs"

In [0]:
cd /content/gdrive/My Drive/{project_directory}/Thaler2017

# Use legacy dependancies

In [0]:
!pip3 install tensorflow==1.0.1
!pip3 install nltk==3.2.5
!pip3 install numpy==1.17.0
!pip3 install matplotlib==2.0.2
!pip3 install scipy==0.19.1
!pip3 install scikit-learn==0.19.0

# SPECIFY dataset name, number (add entry to all_experiments.py)

In [0]:
dataset = "apache"
dataset_number = 18
experiment_type = 'ae_c_2_epochs'

In [0]:
# change dataset/main_dataset_folder name in fiel paths
!cp /content/gdrive/My\ Drive/{project_directory}/Datasets_TB/final_training_log_files/{dataset}/{dataset}.log data
!cp /content/gdrive/My\ Drive/{project_directory}/Datasets_TB/ground_truth/{dataset}/{dataset}.log data/test
!cp /content/gdrive/My\ Drive/{project_directory}/Datasets_TB/ids_files/{dataset}/{dataset}.ids data

In [0]:
# display ipython notebook full width
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
# autoreloading https://ipython.org/ipython-doc/3/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

In [0]:
from library.all_imports import * 
from library.helpers import save_to_json, load_from_json, save_to_csv, create_if_not_exists, get_N_HexCol, multiprocess_file
from library.plotting import plot

assert tf.__version__=="1.0.1" # the tensorflow library we used 
tf.set_random_seed(0)
np.random.seed(0) # fix random seed for reproducability
# tf.logging.set_verbosity(tf.logging.INFO)

# Hyperparameter

In [0]:
from library.all_experiments import * # experiments are defined here

def lib_name(exp_nr):
    if exp_nr in EXPERIMENT_LIB_MAPPING:
        return "data_generation.%s"%EXPERIMENT_LIB_MAPPING[exp_nr]
    else:
        return "data_generation.%s"%ALL_EXPERIMENTS[exp_nr] 
    
experiment_nr = dataset_number
EXPERIMENT_ID = ALL_EXPERIMENTS[experiment_nr] # choose data - it will be automatically generated
print("Running experiment: %s"%EXPERIMENT_ID)
EXPERIMENT_SPLIT_TOKEN = SPLIT_TOKEN[experiment_nr] if (experiment_nr in SPLIT_TOKEN.keys()) else SPLIT_TOKEN["default"]

unknown_token = g.unknown_token = "UNKNOWN_TOKEN"
logline_start_token = g.logline_start_token = "LOG_START"
logline_end_token = g.logline_end_token= "LOG_END"
pad_token = g.pad_token = "PAD_TOKEN"
g.vocabulary_max_lines = -1 # -1 means unlimited
g.max_line_len = 200

# define names
run_tag = "m-pn"
model_name = "rnn-autoencoder"

# clustering 
cluster_alg="birch" # alternatives:  birch hierarch dbscan  

# create experiment directories
result_dir = "results"
experiment_dir = "%.2d_%s"%(experiment_nr,EXPERIMENT_ID)
experiment_outdir = join_path(result_dir, experiment_dir ,  now_str )
checkpoint_path = join_path(experiment_dir , "graph", run_tag)
graph_dir = join_path(experiment_outdir,"graph",run_tag)

create_if_not_exists(checkpoint_path)
create_if_not_exists(graph_dir)

# data files
datafile = g.datafile =  "data/%s.log"%EXPERIMENT_ID
labels_true_file = "data/%s.ids"%EXPERIMENT_ID
processed_datafile = join_path(result_dir, experiment_dir,"%s_log.processed"%EXPERIMENT_ID)
test_data_processed = join_path(result_dir, experiment_dir,"%s_test_log.processed"%EXPERIMENT_ID) 
VOCABULARY_FILE = g.VOCABULARY_FILE = join_path(result_dir, experiment_dir, "vocabulary.json")
HYPERPARAMETERS_FILE = join_path(experiment_outdir, "hyperparams.json")
CLUSTERING_RESULTS_FILE = join_path(experiment_outdir, "clustering_results.csv")
learned_weights_file = join_path(experiment_outdir,"learned_weights.csv")
DATASET_STATS_FILE = join_path(result_dir, experiment_dir, "dataset_statistics.json")                                

g.REGENERATE_VOCABULARY_FILES = True    
g.REPROCESS_TRAININGS_DATA  = True
learn_model = True

# Generate vocabulary

In [0]:
g.WORD_TO_INDEX_FILE = join_path(result_dir, experiment_dir, "word_to_index.json")
g.INDEX_TO_WORD_FILE = join_path(result_dir, experiment_dir, "index_to_word.json")
g.TOKENIZED_LOGLINES_FILE = join_path(result_dir, experiment_dir, "tokenized_loglines.json")
g.SPLIT_TOKEN = EXPERIMENT_SPLIT_TOKEN

# g.SPLIT_TOKEN = ['"']
from library.vocabulary import *
print("Dataset fully loaded")       

# Datset statistics

In [0]:
#for w in sorted(word_frequencies.iteritems(), key= lambda x:x[1], reverse=True):
del word_frequencies["LOG_START"]
del word_frequencies["LOG_END"]

words,frequencies = zip(*sorted(word_frequencies.items(), key= lambda x:x[1], reverse=True))
print("sorted words")
print(words[0:200])

total_words = len(words)
stratify_parts = 200

stratum_length = total_words/stratify_parts

indices = []

for i in range(stratify_parts):    
    strat_start = i* stratum_length
    strat_end = (i+1) * stratum_length -1   
    new_index = np.random.randint(strat_start, strat_end)
#    print(strat_start, strat_end, new_index)
    indices.append(new_index)
    
words = np.array(words)[indices]
frequencies = np.array(frequencies)[indices]

print("Sorted words.")

words = [str(w) for w in words]
colors = []
for w in words:
    if w.isdigit():
        colors.append("r")# "#ffcccc" light red      
    else:
        colors.append("#ccffcc") #light green
print("Defined colors")
        
cluster_plot_file = join_path(result_dir,experiment_dir, "dataset_stats.png")
figure_size=(20,10) 
fig = plt.figure(figsize=figure_size)

index = np.arange(len(words))
bar_width = 0.35


ax1 = fig.add_subplot(111)
ax1.bar(index, frequencies,  color=colors)
ax1.set_yscale('log')
plt.xticks(index + bar_width / 2, (words), rotation=90, fontsize = 8)
plt.savefig(cluster_plot_file)
plt.close()

Image(filename=cluster_plot_file) 

# Average LogLine Length

In [0]:
line_lengths = []
for logline in tokenized_loglines: 
    line_lengths.append(len(logline))

In [0]:
np.average(line_lengths)

# Process trainingsdata

In [0]:
# Create the training data
if g.REPROCESS_TRAININGS_DATA: 
    X_train = np.asarray([[word_to_index[w] for w in logline] for logline in tokenized_loglines])
    maximum_sequence_length =-1
    train_numbers_file = open(processed_datafile,"w")
    one_percent = len(X_train)/100
    for i, logline_as_word_id_sequence in enumerate(X_train):
        if i%one_percent==0: print("Written line %i"%i)
        #print ",".join(line)

        reversed_seq = list(reversed(logline_as_word_id_sequence))
        # add sequence length to example

        sequence_length = str(len(logline_as_word_id_sequence))
        if len(logline_as_word_id_sequence)>maximum_sequence_length: maximum_sequence_length=len(logline_as_word_id_sequence) # find maximum sequence length
        word_id_seq = ",".join(map(str,logline_as_word_id_sequence)) # encoder input: 1,2,3 
        word_id_seq_reversed = ",".join(map(str,reversed_seq)) # decoder input: 3,2,1 
        target_seq = ",".join(map(str, reversed_seq[1:]+[PAD_ID])) # decoder target: 2,1,PAD 

        signature_id =  5 # experiment_lib.extract_pattern_id(loglines[i]) -------- put bogus value---------------
        assert not signature_id==0,"Each log line has to be associated to one signature, none for (%s). Check extract_pattern_id method."%loglines[i]
        # write      
        train_numbers_file.write("%s|%s|%s|%s|%s\n"%(signature_id, sequence_length, word_id_seq,word_id_seq_reversed,target_seq))

    train_numbers_file.close()

    ###

    #### tokenizing test data part
    tokenized_test_loglines = []

    test_loglines = list(open("data/test/%s.log"%EXPERIMENT_ID, 'r'))
    print("Loaded %i test_loglines"%len(test_loglines))
    total_lines = len(test_loglines) if g.vocabulary_max_lines==-1 else g.vocabulary_max_lines
    print("Tokenizing test %i lines... "%total_lines)
    print("test_loglines",len(test_loglines))
    
    for i, test_logline in enumerate(test_loglines):
        
        test_logline = test_logline.lower()
        
        for char in g.SPLIT_TOKEN:
            test_logline = test_logline.replace(char, ' ' + char + ' ')
        
        tokenized_test_logline = test_logline.split(" ")[0:200] #nltk.word_tokenize(logline)
        
        tokenized_test_logline = [g.logline_start_token] + tokenized_test_logline + [g.logline_end_token]
        
        tokenized_test_loglines.append(tokenized_test_logline)

        if g.vocabulary_max_lines>0 and i>g.vocabulary_max_lines:
            break
    
    for i, test_logline in enumerate(tokenized_test_loglines):
        tokenized_test_loglines[i] = [w if w in word_to_index else g.unknown_token for w in test_logline]

    print ("Tokenized logfile.")
    #### tokeninzing test data part finish

    array_to_xtrain = []

    X_test = np.asarray([[word_to_index[w] for w in logline] for logline in tokenized_test_loglines])

    maximum_sequence_length =-1
    test_numbers_file = open(test_data_processed,"w")
    one_percent = len(X_test)/100

    for i, test_logline_as_word_id_sequence in enumerate(X_test):

        reversed_seq = list(reversed(test_logline_as_word_id_sequence))
        # add sequence length to example

        sequence_length = str(len(test_logline_as_word_id_sequence))
        if len(test_logline_as_word_id_sequence)>maximum_sequence_length: maximum_sequence_length=len(test_logline_as_word_id_sequence) # find maximum sequence length
        word_id_seq = ",".join(map(str,test_logline_as_word_id_sequence)) # encoder input: 1,2,3 
        word_id_seq_reversed = ",".join(map(str,reversed_seq)) # decoder input: 3,2,1 
        target_seq = ",".join(map(str, reversed_seq[1:]+[PAD_ID])) # decoder target: 2,1,PAD 

        # signature_id = experiment_lib.extract_pattern_id(loglines[i]) #######
        signature_id = 5

        assert not signature_id==0,"Each log line has to be associated to one signature, none for (%s). Check extract_pattern_id method."%loglines[i]
        # write      
        test_numbers_file.write("%s|%s|%s|%s|%s\n"%(signature_id, sequence_length, word_id_seq,word_id_seq_reversed,target_seq))
    test_numbers_file.close()
    ###
else:
    X_train = np.asarray([[word_to_index[w] for w in logline] for logline in tokenized_loglines])
    print("Trainingsdata has already been processed")

if not os.path.exists(labels_true_file):
    os.system("python3 create_true_labels.py -en %s"%EXPERIMENT_ID)

#  Graph Hyper parameters

In [0]:
# TDB save vocabulary file
state_size = 256 # size of vector internal of a cell
batch_size = 200

num_examples_to_visualize = min(10000, len(tokenized_loglines)) # how many dots to show
num_examples_to_embed = len(tokenized_test_loglines) #len(tokenized_loglines) -------------changed according to the test set------------

dropout_keep_probability = 0.7
num_lstm_layers = 1

DTYPE=tf.float32
num_samples = min(vocabulary_size, 500) # number of samples to draw for sampled softmax
max_gradient_norm = 0.5 # to be defined

LEARNING_RATE = 0.02
learning_rate_decay_factor = 0.95
l1_scale= 0.000
epochs = 1

num_examples = len(X_train)
max_steps = int(epochs * (num_examples / batch_size))
learning_rate_adjustments = 10 # how many times to apply learning rate decay over all epochs
adjust_learning_rate_after_steps = max_steps / learning_rate_adjustments

# some tf varioables
tf_keep_probabiltiy = tf.constant(dropout_keep_probability) 
tf_global_step = tf.Variable(0, trainable=False)


# clustering hierarchy
examples_in_hierarchy = 50
color_threshhold = 0.2

# threshold for vmeasure, homogenity etc.
h_threshold = 0.004

# save hyperparameters
hyperparams = {
    "state_size":state_size,
    "num_examples_to_visualize":num_examples_to_visualize,
    "dropout_keep_probability":dropout_keep_probability,
    "num_lstm_layers":num_lstm_layers,
    "learning_rate_decay_factor":learning_rate_decay_factor,
    "batch_size":batch_size,
    "dtype":str(DTYPE),
    "num_samples":num_samples,
    "max_gradient_norm":max_gradient_norm,
    "learning_rate":LEARNING_RATE,
    "epochs":epochs,
    "num_examples":num_examples,
    "max_steps":max_steps,
    "examples_in_hierarchy":examples_in_hierarchy,
    "color_threshhold":color_threshhold,    
    "h_threshold":h_threshold,
}


save_to_json(hyperparams, HYPERPARAMETERS_FILE)

#  Graph

## Learning Rate

In [0]:
learning_rate = tf.train.exponential_decay(
  learning_rate=LEARNING_RATE, 
  global_step=tf_global_step, # current learning step
  decay_steps= adjust_learning_rate_after_steps, # how many steps to train after decaying learning rate      
  decay_rate=learning_rate_decay_factor,                
  staircase=True)


## Input, Output and Target of the graph

In [0]:

# inputs, outputs
x_e = tf.placeholder(tf.int32, [batch_size, None]) # encoder inputs loglines [batch_size, num_steps]
x_d = tf.placeholder(tf.int32, [batch_size, None]) # decoder inputs 
y_d = tf.placeholder(tf.int32, [batch_size, None]) # reversed loglines [batch_size, num_steps]

visualization_embeddings = tf.Variable(np.zeros([num_examples_to_visualize, state_size]) , trainable=False, name="VisualizationEmbeddings", dtype= DTYPE   )

word_embeddings = tf.get_variable('word_embeddings', [vocabulary_size, state_size], dtype= DTYPE ) # each row is a dense vector for each word.

# Encoder Inputs
encoder_inputs = tf.nn.embedding_lookup(word_embeddings, x_e) # [batch_size, max_time, embedding_size]
encoder_sequence_lengths = tf.placeholder(tf.int32, [batch_size]) # [batch_size]

# Decoder Inputs
decoder_inputs = tf.nn.embedding_lookup(word_embeddings, x_d) # need to be defined  [batch_size, max_time, embedding_size]
decoder_sequence_lengths = tf.placeholder(tf.int32, [batch_size]) # [batch_size]

# Decoder Labels
decoder_labels = y_d # this are our target words

## LSTM, Dropout Wrapper, MultiRNNCell

In [0]:
from library.core_rnn_cell_impl import DropoutWrapper as DtypeDropoutWrapper # import v1.1.0 dropout wrapper to support setting DTYPE to half-precision

# Define cells
with tf.variable_scope("encoder_scope") as encoder_scope:

    cell = contrib_rnn.LSTMCell(num_units=state_size, state_is_tuple=True)
    cell = DtypeDropoutWrapper(cell=cell, output_keep_prob=tf_keep_probabiltiy, dtype=DTYPE)
    cell = contrib_rnn.MultiRNNCell(cells=[cell] * num_lstm_layers, state_is_tuple=True)

    encoder_cell = cell # needs to be defined

    # encode inputs
    encoder_outputs, last_encoder_state = tf.nn.dynamic_rnn(
        cell=encoder_cell,
        dtype=DTYPE,
        sequence_length=encoder_sequence_lengths,
        inputs=encoder_inputs,
        )

## Dynamic RNN decoder

In [0]:
with tf.variable_scope("decoder_scope") as decoder_scope:
        # output projection
    # we need to specify output projection manually, because sampled softmax needs to have access to the the projection matrix 
    output_projection_w_t = tf.get_variable("output_projection_w", [vocabulary_size, state_size], dtype=DTYPE)
    output_projection_w = tf.transpose(output_projection_w_t)
    output_projection_b = tf.get_variable("output_projection_b", [vocabulary_size], dtype=DTYPE)
    
    # define decoder cell
    decoder_cell = tf.contrib.rnn.LSTMCell(num_units=state_size)
    decoder_cell = DtypeDropoutWrapper(cell=decoder_cell, output_keep_prob=tf_keep_probabiltiy, dtype=DTYPE)
    decoder_cell = contrib_rnn.MultiRNNCell(cells=[decoder_cell] * num_lstm_layers, state_is_tuple=True)   
    # decoder_cell = contrib_rnn.OutputProjectionWrapper(decoder_cell, output_size=vocabulary_size ) # rnn output: [batch_size, max_time, vocabulary_size ]

    # define decoder train netowrk
    decoder_outputs_tr, _ , _ = dynamic_rnn_decoder( # decoder outputs, final hidden state, final context state
        cell=decoder_cell, # the cell function
        decoder_fn= simple_decoder_fn_train(last_encoder_state, name=None),
        inputs=decoder_inputs, # [batch_size, max_time, embedding_size].
        sequence_length=decoder_sequence_lengths, #  length for sequence in the batch [batch_size] 
        parallel_iterations=None, # Tradeoff - time for memory
        swap_memory=False,
        time_major=False)
    
    # define decoder inference network
    decoder_scope.reuse_variables()    
    

# Objective and Optimization


## Loss function

In [0]:
print("Decoder Output  Shape: [batch_size=%s, max_timesteps=%s, vocabulary_size=%s]"%tuple(decoder_outputs_tr.shape))
#from library.nn_impl import sampled_softmax_loss as dtype_sampled_softmax_loss # imported cutom loss function to support dtype  as a parameter => half precision

# reshape outputs of decoders to [ batch_size * max_time , vocabulary_size ]
decoder_forward_outputs = tf.reshape(decoder_outputs_tr,[-1, state_size])
decoder_target_labels  = tf.reshape(decoder_labels ,[-1, 1])# =>  [ batch_size * max_time ]  sequence of correc tlabel

sampled_softmax_losses = tf.nn.sampled_softmax_loss(
    weights = output_projection_w_t,# [num_classes, state_size]
    biases = output_projection_b, # [num_classes]
    inputs = decoder_forward_outputs, #  inputs: A `Tensor` of shape `[batch_size, state_size]`.  The forward activations of the input network. 
    labels = decoder_target_labels , 
    num_sampled = num_samples,
    num_classes=vocabulary_size,
    num_true = 1,
)    
total_loss_op = tf.reduce_mean(sampled_softmax_losses) 


## Regularization

In [0]:
l1_regularizer = tf.contrib.layers.l1_regularizer(
    scale=l1_scale, scope=None
)
weights = tf.trainable_variables()
regularization_penalty = tf.contrib.layers.apply_regularization(l1_regularizer, [word_embeddings])

regularized_loss = total_loss_op + regularization_penalty

## Trainings step

In [0]:
# get gradients for all trainable parameters with respect to our loss funciton
params = tf.trainable_variables()
#optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
gradients = tf.gradients(regularized_loss, params)

# apply gradient clip
clipped_gradients, gradient_norm = tf.clip_by_global_norm(gradients,max_gradient_norm)

# Update operation
training_step  = optimizer.apply_gradients(zip(clipped_gradients, params), global_step=tf_global_step) # learing rate decay is calulated based on this step

# Training 

## Prepare Batch Loading for trainings data

In [0]:
# load single example from processed file
filename_queue = tf.train.string_input_producer([processed_datafile], num_epochs=None) # num_epochs=None 
reader = tf.TextLineReader()
example_id, single_example_logline = reader.read(filename_queue) 

In [0]:
# define how to parse single example (see 1.3)
split_example = tf.string_split([single_example_logline],delimiter="|")
split_example_dense = tf.sparse_tensor_to_dense(split_example, default_value='', validate_indices=True, name=None)
split_example_dense = split_example_dense[0]

In [0]:
# split_example_dense[0] is signature id
# sequence length
sequence_length=tf.string_to_number(split_example_dense[1],out_type=tf.int32)
# split encoder inputs
enc_split =  tf.string_split([split_example_dense[2]],delimiter=",")
enc_split_dense = tf.sparse_tensor_to_dense(enc_split, default_value='', validate_indices=True, name=None)
x_e_single = tf.string_to_number(enc_split_dense, out_type=tf.int32) 
# split decoder inputs
dec_split =  tf.string_split([split_example_dense[3]],delimiter=",")
dec_split_dense = tf.sparse_tensor_to_dense(dec_split, default_value='', validate_indices=True, name=None)
x_d_single = tf.string_to_number(dec_split_dense, out_type=tf.int32) 
# split decoder targets
tar_split =  tf.string_split([split_example_dense[4]],delimiter=",")
tar_split_dense = tf.sparse_tensor_to_dense(tar_split, default_value='', validate_indices=True, name=None)
y_single = tf.string_to_number(tar_split_dense, out_type=tf.int32)

In [0]:
# Batch the variable length tensor with dynamic padding
fetch_trainings_batch = tf.train.batch(
    tensors=[
        x_e_single[0], #single encoder input line
        x_d_single[0], #single decoder input line 
        y_single[0], #single decoder target line  
        sequence_length,   # sequence length of this example
    ],
    batch_size=batch_size,
    dynamic_pad=True,
    name="trainings_batch",
    allow_smaller_final_batch=True
)

# Session Setup

In [0]:
def get_batch_dict(session):
    # get trainingsbatch from input queue
    batch = session.run([fetch_trainings_batch], feed_dict=None)
    batch = batch[0] 
        
    # assign arrays to dictionary
    batch_dict = {
        x_e:batch[0], # encoder inputs 
        x_d:batch[1], # decoder inputs
        y_d:batch[2], # decoder targets
        encoder_sequence_lengths:batch[3], 
        decoder_sequence_lengths:batch[3]           
    }
    return batch_dict

# Trainings Loop

In [0]:
if learn_model:    
    current_epoch = 0
    save_checkpoint_after_each_step = int(max_steps/10)
    print_loss_after_steps = int(max_steps/100)

    queue_capacity = 2 * batch_size

    # Saver
    saver = tf.train.Saver(tf.global_variables())

    # Summaries
    tf.summary.scalar("total_loss",tf.cast(total_loss_op, DTYPE)) # summary for accuracy
    tf.summary.scalar("regularized_loss",tf.cast(regularized_loss, DTYPE)) # summary for accuracy
    # Start session
    session = tf.Session()

    # Check random seed for reproducability
    control_random_number = session.run(tf.random_normal([1])) 
    print("Control Random Number: %0.5f"%control_random_number) # should change if you modify the graph

    
    all_summaries = tf.summary.merge_all()
    summary_writer = tf.summary.FileWriter(graph_dir, graph=session.graph)

    session.run([
            tf.local_variables_initializer(),
            tf.global_variables_initializer(),
        ])


In [0]:
s= time.time() # start time
if learn_model:

    word_embeddings_before = session.run(word_embeddings)
    import collections
    batch_times = collections.deque()


    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=session, coord=coord)

    for current_step in range(1,max_steps+1): # start from 1 .. max_steps+1 to execute max steps            

        try:
            step_s = time.time()
            # increase step counter
            session.run(tf_global_step.assign(current_step)) 

            # get next batch
            batch_dict=get_batch_dict(session)
            # execute actions
            results = session.run([
                total_loss_op, # calculate training loss
                regularized_loss, #     
                training_step, # calculate gradients, update gradients
                all_summaries, # compile summaries and write to graph dir,
                learning_rate,
                 # get gradients    
            ], feed_dict = batch_dict )

            summary_writer.add_summary(results[3],current_step)
            batch_times.append( (time.time()-step_s))

        except Exception as e:
            # Report exceptions to the coordinator.
            print("Aborting trainingsloop :%s"%str(e.message))
            coord.request_stop(e)
            break

        if current_step % print_loss_after_steps==0:
            print("Epoch {epoch:02d}, Step {current_step:05d}/{max_steps:05d}, Current Learning rate: {learning_rate:0.4f},  Loss: {loss:0.4f} Regularized Loss: {regularized_loss:0.4f}".format(
                    epoch=int(current_step/(max_steps/epochs)+1), 
                    current_step=current_step, max_steps=max_steps,
                    learning_rate=results[4], 
                    loss=results[0],regularized_loss=results[1]
                )# end format
            ) # end print
            avg_batch_time =  sum(batch_times) /len(batch_times)
            total_time_in_s = avg_batch_time * max_steps
            print("Average step time: %0.2fs, Estimated total duration: ~%0.2f min (~ %0.2f h) "% (avg_batch_time, total_time_in_s/60.0, total_time_in_s/3600.0 ))

        # save checkpoint every xth step
        if current_step % save_checkpoint_after_each_step==0:
            print ("Saving checkpoint")

            chkpoint_out_filename = join_path(checkpoint_path, model_name)
            saver.save(session, chkpoint_out_filename , global_step=current_step)             

    # stop training queues
    coord.request_stop()    
    word_embeddings_after = session.run(word_embeddings)
    e = time.time()
    print("Learning took {sec:0.2f} seconds".format(sec=(e-s)))
else:
    current_step = 35821 # TODO this needs to be adjusted 
    
    session = tf.Session()
    meta_data = join_path(result_dir, checkpoint_path,'rnn-autoencoder-35821.meta') # TODO this needs to be adjusted 
    print(meta_data)
    saver = tf.train.Saver()#tf.train.import_meta_graph(meta_data)
    print("Restored graph")
    model_to_load  =  tf.train.latest_checkpoint( join_path( result_dir, checkpoint_path))
    print("Load '%s' from saved checkpoints"%model_to_load)            
    saver.restore(session, model_to_load)

# Visualization


# Embedd test loglines 2000

In [0]:
embedded_csv = join_path(result_dir, experiment_dir,"embedded_lines.csv")
embedded_loglines = np.ndarray(shape=[num_examples_to_embed, state_size])

if not os.path.exists(embedded_csv):
    # load 
    test_examples = list(open(test_data_processed,"r")) 
    print(len(test_examples))
    test_examples = [t.split("|") for t in test_examples]
    print("Loaded trainingsdata")

    s = time.time()
    count = 0
    train_batches  =  list(range(0, num_examples_to_embed, batch_size))
    print("Started embedding %i batches of %i log lines"%(len(train_batches),num_examples_to_embed ))
    for i, ex_id in enumerate(train_batches):
        x_e_batch = []
        esl_batch = []

        incomplete_batch = False # for last batch
        incomplete_length = 0
        # get batch, write metadata
        for b_id, example in enumerate(test_examples[ex_id:ex_id+batch_size]):
            count+=1
            esl_batch.append(int(example[1])) # sequence length
            x_e_batch.append(np.array(example[2].split(","),dtype=np.integer) ) # encoder input
            
        if len(x_e_batch)<batch_size: # last_batch, so need to zero add id
            incomplete_batch=True
            incomplete_length = len(x_e_batch)

            for _ in range(batch_size-len(x_e_batch)):
                esl_batch.append(0)
                x_e_batch.append(np.zeros([len(x_e_batch[0])]))

        # pad batch
        max_seq_len = np.amax(esl_batch) # maximum encoder length
        padded_x_e_batch = []
        for x_es in x_e_batch:
            padded_x_e = np.pad(x_es , (0,max_seq_len-len(x_es)), 'constant', constant_values=0)
            padded_x_e_batch.append(padded_x_e)                          

        #assign arrays to dictionary
        batch_dict = {
            x_e:np.array(padded_x_e_batch), # encoder inputs  # [batch_size, max_sequence_length, state_size ]
            encoder_sequence_lengths:np.array(esl_batch) # encoder sequence length [batch_size,1]           
        }

        # execute actions
        results = session.run([
            last_encoder_state, # get encoded trainings samples
        ], feed_dict = batch_dict )

        # story in temporary array
        if incomplete_batch:
            embedded_loglines[ex_id:ex_id+incomplete_length,] = results[0][0].c[0:incomplete_length]
        else:
            embedded_loglines[ex_id:ex_id+batch_size,] = results[0][0].c # copy c hidden state to tmp_ary


    print("Embedded %i lines in ~%0.2f min"%(num_examples_to_embed, ((time.time()-s )/60.0)))
    a = np.ndarray((num_examples_to_embed,state_size), dtype=float, buffer=None, offset=0,)
    print("shape ", a.shape)
    print("Saved embedded lines to file: %s "%embedded_csv)
    np.savetxt(embedded_csv, a, delimiter=",")
else:
    embedded_loglines = np.genfromtxt(embedded_csv, delimiter=";")
    print("Loaded embedded lines to file: %s "%embedded_csv)
    print(embedded_loglines.shape)

X = embedded_loglines
for i in X:
  print(i)

# Clustering


In [0]:
from sklearn.cluster import Birch
cluster_alg="birch"

In [0]:
labels_true_file = "/content/gdrive/My Drive/"+ project_directory + "/Thaler2017/data/" + dataset + ".ids"
print(labels_true_file)

In [0]:
f=open(labels_true_file, "r")  ###path of the predicted label array consisting text file should be mentioned
contents = f.read()
f.close()
mod_contents = re.findall(r"[\w']+",contents)
test_list = [int(i) for i in mod_contents]

np.array(list(test_list))

In [0]:
clusters = np.array(list(test_list))
clusters

In [0]:
len(clusters)

In [0]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import Birch
from sklearn.metrics import homogeneity_completeness_v_measure, silhouette_score, mutual_info_score, adjusted_mutual_info_score
import re
import sys
from collections import defaultdict
import scipy.special
from itertools import combinations

In [0]:
def getClusters(arr):
    clusters = []
    c = np.unique(arr)
    for i in c:
        res_list = list(filter(lambda x: arr[x] == i, range(len(arr))))
        clusters += [res_list]
    return clusters

# def f1Score(trueLabels, predictedLabels):
#     true_labels = getClusters(trueLabels)
#     predicted_labels = getClusters(predictedLabels)
#     dictA = []
#     dictB = []
#     for i in true_labels:
#         comb = combinations(i, 2)
#         for j in list(comb):
#             dictA.append(list(j))

#     for i in predicted_labels:
#         comb = combinations(i, 2)
#         for j in list(comb):
#             dictB.append(list(j))

#     a = len([list(x) for x in set(tuple(x) for x in dictA).intersection(set(tuple(x) for x in dictB))])
#     b = len([x for x in dictB if x not in dictA])
#     c = len([x for x in dictA if x not in dictB])
#     f1_score = (2 * a) / ((2 * a) + b + c)
#     return f1_score

In [0]:
X

In [0]:
min_thres = 0.1
max_thres = 30
thres_step = 0.2

predicted_labels = {}
cluster_results = {}

for i in np.arange (min_thres,max_thres,thres_step):
  row = []
  if (cluster_alg=="birch"):
    brc = Birch(threshold=i, branching_factor=50, n_clusters=None, compute_labels=True, copy=True)
    brc.fit(X)
    labels = brc.predict(X)
  elif cluster_alg=="dbscan":
    print("dbscan")
    break
  
  
  hom_com_vmes = homogeneity_completeness_v_measure(clusters, labels)
  
  shilloutte = silhouette_score(X, clusters)
  mutualinfoscore = mutual_info_score(clusters, labels, contingency=None)
  adjustedmutualinfoscore = adjusted_mutual_info_score(clusters, labels)
  # f1_score = f1Score(clusters,labels)
  
  row+=list(hom_com_vmes)
  row.append(shilloutte)
  # row.append(f1_score)
  row.append(mutualinfoscore)
  row.append(adjustedmutualinfoscore)
  try:
    predicted_labels[i].append(labels)
  except KeyError:
    predicted_labels[i] = labels
  cluster_results[i] = row
print(predicted_labels)
print(cluster_results)

In [0]:
print(len(cluster_results))

In [0]:
for p_l in predicted_labels:
  print(p_l, len(np.unique(predicted_labels[p_l])), cluster_results[p_l][2])

In [0]:
import matplotlib.pyplot as plt 

homogenity = []
completeness = []
vScore = []
shilloutte = []
f_score = []
mutualinfoscore =[]
adjustedmutualinfoscore = []

y = list(cluster_results.keys())
print("Threshold : ", list(y))
for val in cluster_results.items():
  homogenity+=[val[1][0]]
  completeness+=[val[1][1]]
  vScore+=[val[1][2]]
  shilloutte+=[val[1][3]]
  # f_score+=[val[1][4]]
  mutualinfoscore+=[val[1][4]]
  adjustedmutualinfoscore+=[val[1][5]]

# print("F1 Score : ", f_score)
print("Homogenity : ", homogenity)
print("Completeness : ", completeness)
print("V-Score : ", vScore)
print("Shilloutte", shilloutte)
print("Mutual info score", mutualinfoscore)
print("Adjusted mutual info score", adjustedmutualinfoscore)

# plt.plot(y,f_score, label = "F1 Score")
plt.plot(y,homogenity, label = "Homogenity")
plt.plot( y,completeness, label = "Completeness")
plt.plot( y,vScore, label = "V-Score")
plt.plot(y,shilloutte, label = "Shilloutte")
plt.plot(y,mutualinfoscore, label = "Mutual_info_score")
plt.plot(y,adjustedmutualinfoscore, label = "Adjusted mutual info score")

plt.xlabel('Threshold') 

plt.legend()

plt.show() 
print("Highest V-cscore :", y[vScore.index(max(vScore))])

# Write Clustering Results

In [0]:
clustering_Results = {
        'Threshold': y,
        'Homogenity': homogenity,
        'Completeness': completeness,
        'vScore': vScore,
        'Shilloutte': shilloutte,
        'Mutual info score': mutualinfoscore,
        'Adjusted mutual info score': adjustedmutualinfoscore
    }

results = pd.DataFrame(clustering_Results)

results.to_csv(join_path(result_dir, experiment_dir, dataset + '_clustering_results.csv'))

# Get template

In [0]:
def getTemplate(candidate):
    # candidate: list of list
    abstraction = ''

    # transpose row to column
    candidate_transpose = list(zip(*candidate))
    candidate_length = len(candidate)

    if candidate_length > 1:
        # get abstraction
        abstraction_list = []
        for index, message in enumerate(candidate_transpose):
            message_length = len(set(message))
            if message_length == 1:
                abstraction_list.append(message[0])
            else:
                abstraction_list.append('<*>')

        abstraction = ' '.join(abstraction_list)

    elif candidate_length == 1:
        abstraction = ' '.join(candidate[0])

    return abstraction

# Read Ground ruth templates

In [0]:
import pandas as pd
df_raw_logs = pd.read_csv("/content/gdrive/My Drive/" + project_directory + "/Datasets_TB/ground_truth/"+ dataset + "/" + dataset +"_2k.log_structured.csv") 
raw_logs = df_raw_logs['Content'].to_numpy()

In [0]:
raw_logs

# Get best clusterong based config

In [0]:
len(raw_logs)

In [0]:
predicted_labels = predicted_labels[y[vScore.index(max(vScore))]]
predicted_labels

In [0]:
groups = pd.DataFrame()
groups['predicted_labels'] = predicted_labels
groups['raw_logs'] = raw_logs
groups

In [0]:
for i in np.unique(groups.predicted_labels):
  print(i)
  for index,row in groups.iterrows():
    if row['predicted_labels'] == i:
      print(row['raw_logs'])
  print('\n')


In [0]:
clusters = {}
for i in range(0, len(predicted_labels)):
    try:
        clusters[predicted_labels[i]].append(raw_logs[i].split(" "))
    except KeyError:
        clusters[predicted_labels[i]] = [raw_logs[i].split(" ")]


output = {}
for cluster in clusters.items():
  template = getTemplate(cluster[1])
  output['E' + str(cluster[0])] = template
  print(template)
  for log in cluster[1]:
    print(log)
  print('\n')


def writeOutput(output):
    line = {
        'EventId': list(output.keys()),
        'EventTemplate': list(output.values())
    }

    df = pd.DataFrame(line)
    df.to_csv(join_path(result_dir, experiment_dir, dataset + '_templates.csv'))
  
  
writeOutput(output)

In [0]:
structured_log = df_raw_logs.drop(['EventTemplate','EventId'], axis=1)
df_eventTemplates = pd.read_csv(join_path(result_dir, experiment_dir, dataset + '_templates.csv'))

In [0]:
template = []
eventId = []
 
for label in predicted_labels:
  template.append(df_eventTemplates[df_eventTemplates['EventId'] == 'E'+str(label)]['EventTemplate']) 
  eventId.append(df_eventTemplates[df_eventTemplates['EventId'] == 'E'+str(label)]['EventId'])

In [0]:
structured_log['EventId'] = eventId
structured_log['EventTemplate'] = template

In [0]:
structured_log.to_csv(join_path(result_dir, experiment_dir, dataset + '_structured.csv'))

In [0]:
def evaluate(groundtruth, parsedresult):
    
    df_groundtruth = pd.read_csv(groundtruth)
    df_parsedlog = pd.read_csv(parsedresult)

    # Remove invalid groundtruth event Templates
    null_logids = df_groundtruth[~df_groundtruth['EventTemplate'].isnull()].index
    df_groundtruth = df_groundtruth.loc[null_logids]
    df_parsedlog = df_parsedlog.loc[null_logids]

    (precision, recall, f_measure, accuracy) = get_accuracy(df_groundtruth['EventTemplate'], df_parsedlog['EventTemplate'])
    print('Precision: %.4f, Recall: %.4f, F1_measure: %.4f, Parsing_Accuracy: %.4f' % (
        precision, recall, f_measure, accuracy))
    return f_measure, accuracy


def get_accuracy(series_groundtruth, series_parsedlog, debug=False):
   
    series_groundtruth_valuecounts = series_groundtruth.value_counts()
    real_pairs = 0
    for count in series_groundtruth_valuecounts:
        if count > 1:
            real_pairs += scipy.special.comb(count, 2)

    series_parsedlog_valuecounts = series_parsedlog.value_counts()
    parsed_pairs = 0
    for count in series_parsedlog_valuecounts:
        if count > 1:
            parsed_pairs += scipy.special.comb(count, 2)

    accurate_pairs = 0
    accurate_events = 0  # determine how many lines are correctly parsed
    for parsed_eventId in series_parsedlog_valuecounts.index:
        logIds = series_parsedlog[series_parsedlog == parsed_eventId].index
        series_groundtruth_logId_valuecounts = series_groundtruth[logIds].value_counts()
        error_eventIds = (parsed_eventId, series_groundtruth_logId_valuecounts.index.tolist())
        error = True
        if series_groundtruth_logId_valuecounts.size == 1:
            groundtruth_eventId = series_groundtruth_logId_valuecounts.index[0]
            if logIds.size == series_groundtruth[series_groundtruth == groundtruth_eventId].size:
                accurate_events += logIds.size
                error = False
        if error and debug:
            print('(parsed_eventId, groundtruth_eventId) =', error_eventIds, 'failed', logIds.size, 'messages')
        for count in series_groundtruth_logId_valuecounts:
            if count > 1:
                accurate_pairs += scipy.special.comb(count, 2)

    precision = float(accurate_pairs) / parsed_pairs
    recall = float(accurate_pairs) / real_pairs
    f_measure = 2 * precision * recall / (precision + recall)
    accuracy = float(accurate_events) / series_groundtruth.size
    return precision, recall, f_measure, accuracy


In [0]:
print(evaluate("/content/gdrive/My Drive/" + project_directory + "/Datasets_TB/ground_truth/"+ dataset + "/" + dataset +"_2k.log_structured.csv",    #####path of the structured log csv of the groundtruth
               join_path(result_dir, experiment_dir, dataset + '_structured.csv')))

In [0]:
cd ..

In [0]:
!cp -r Thaler2017/results/{dataset_number}_{dataset} /content/gdrive/My\ Drive/{project_directory}/our_experiments/{experiment_type}/{dataset}